In [1]:
# Jupyter notebook setup
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Standard imports
import sys
import os
from pathlib import Path

# Add project root to path if needed
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

In [2]:
import matplotlib.pyplot as plt
from os import path

from tqdm import tqdm
import torch
from omegaconf import OmegaConf, open_dict

from experanto.datasets import ChunkDataset
from experanto.dataloaders import get_multisession_dataloader

from nexport.exporters.utils.storage import save_dataset_to_folder

In [3]:
from experanto.configs import DEFAULT_CONFIG as cfg

print(OmegaConf.to_yaml(cfg))

dataset:
  global_sampling_rate: null
  global_chunk_size: null
  add_behavior_as_channels: false
  replace_nans_with_means: false
  cache_data: false
  out_keys:
  - screen
  - responses
  - eye_tracker
  - treadmill
  - timestamps
  normalize_timestamps: true
  modality_config:
    screen:
      keep_nans: false
      sampling_rate: 30
      chunk_size: 60
      valid_condition:
        tier: train
      offset: 0
      sample_stride: 1
      include_blanks: true
      transforms:
        normalization: normalize
        Resize:
          _target_: torchvision.transforms.v2.Resize
          size:
          - 144
          - 256
      interpolation:
        rescale: true
        rescale_size:
        - 144
        - 256
    responses:
      keep_nans: false
      sampling_rate: 8
      chunk_size: 16
      offset: 0.0
      transforms:
        normalization: standardize
      interpolation:
        interpolation_mode: nearest_neighbor
      filters:
        nan_filter:
          __tar

In [4]:
cfg.dataset.modality_config.screen.include_blanks = True
cfg.dataset.modality_config.screen.valid_condition = {"tier": "train"}
cfg.dataloader.num_workers = 8

In [5]:
from experanto.dataloaders import get_multisession_dataloader

paths = ["/data/dynamic29513-3-5-Video-full"]
train_dl = get_multisession_dataloader(paths, cfg)

No metadata file found at /data/dynamic29513-3-5-Video-full/meta.json


/usr/local/lib/python3.12/dist-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:626: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [6]:
from experanto.experiment import Experiment
from experanto.configs import DEFAULT_MODALITY_CONFIG

# Define the path to your experiment data
root_folder = '/data/dynamic29513-3-5-Video-full'

# Initialize the experiment
experiment = Experiment(root_folder, modality_config=DEFAULT_MODALITY_CONFIG)

In [7]:
import numpy as np
from experanto.intervals import TimeInterval

# Access the screen device to find the timestamps of the first 10 frames
screen_device = experiment.devices['screen']

# Assuming screen timestamps are available and sorted
# Get the start and end time for the first 10 frames
# Note: This logic assumes frame 0 is the start. Adjust indices if necessary.
start_time = screen_device.timestamps[0]
end_time = screen_device.timestamps[10] # or 9, depending on if you want inclusive/exclusive

# Create a TimeInterval object
interval = TimeInterval(start_time, end_time)

In [8]:
# Define the sampling rates you want for the output (optional, defaults to config)
# You might want to match the screen's native rate or a fixed rate like 30Hz
target_rates = {
    'screen': 30.0,      # Example: 30 Hz
    'responses': 30.0,   # Match screen rate for easy alignment, or keep native
    # 'behaviors': 30.0    # Match screen rate
    'eye_tracker': 30.0,  # Add specific device names found in your experiment
    'treadmill': 30.0
}

# Fetch the data
# This returns a dictionary with keys like 'screen', 'responses', etc.
subset_data = experiment.get_data_for_interval(
    interval,
    target_sampling_rates=target_rates
)

In [12]:
# Create the dictionary for the exporter
export_data = {
    'screen': subset_data['screen'],
    'responses': subset_data['responses'],  # Keep 'responses'
    'behavior': subset_data.get('eye_tracker'), # Map 'behaviors' (if it exists)
    'behavior': subset_data.get('treadmill'), # Map 'behaviors' (if it exists)
    # Add other keys if necessary, like 'pupil_center' if available
}

# 2. Add Metadata
# Example: Adding trial tiers (e.g., 'train', 'test', 'validation')
# Since we have 10 frames, we need 10 entries if this is per-frame metadata,
# or it might be per-trial depending on how you define a "trial".
# Assuming 1-to-1 mapping with the 'screen' dimension:
export_data['tiers'] = np.array(['train'] * 10) 

# Example: Adding Neuron Metadata (e.g., cell locations, ids)
# This should match the second dimension of your 'responses' (n_neurons)
n_neurons = subset_data['responses'].shape[1]
export_data['neurons'] = {
    'cell_ids': np.arange(n_neurons),
    'cell_types': np.array(['excitatory'] * n_neurons) 
}

# This dictionary is expected by save_to_folder.
export_data['item_info'] = {}

# Example: Adding Statistics (e.g., mean/std of the original data)
export_data['statistics'] = {
    'responses': {
        'mean': np.mean(subset_data['responses'], axis=0),
        'std': np.std(subset_data['responses'], axis=0)
    }
}

# 3. Save the dataset including metadata
# Note: You don't need to add 'tiers', 'neurons', etc. to data_keys 
# if you use the default logic, but it's safer to be explicit if you are filtering.
# However, `save_to_folder` has hardcoded checks for these specific meta keys 
# separate from the `data_keys` loop.
save_dataset_to_folder(
    data=export_data,
    full_path='./my_dataset_with_meta',
    overwrite=True,
    # data_keys usually controls the "data" folder content. 
    # Metadata keys are handled automatically if present in the dictionary.
    data_keys=['screen', 'responses'] 
)

Saving responses: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 239.07it/s]

11-12-2025:15:47:52 INFO     storage.py            84:	 Saving my_dataset_with_meta/meta/trials/tiers.npy
11-12-2025:15:47:52 INFO     storage.py            84:	 Saving my_dataset_with_meta/meta/neurons/cell_ids.npy
11-12-2025:15:47:52 INFO     storage.py            84:	 Saving my_dataset_with_meta/meta/neurons/cell_types.npy
11-12-2025:15:47:52 INFO     storage.py            84:	 Saving my_dataset_with_meta/meta/statistics/responses/mean.npy
11-12-2025:15:47:52 INFO     storage.py            84:	 Saving my_dataset_with_meta/meta/statistics/responses/std.npy


In [13]:
import sys
import os
from pathlib import Path
from omegaconf import OmegaConf

# Import experanto modules
from experanto.configs import DEFAULT_CONFIG as cfg
from experanto.dataloaders import get_multisession_dataloader

# 1. Define the path to your exported dataset
# This matches the 'full_path' used in the previous save_dataset_to_folder step
dataset_path = str(Path('./my_dataset_with_meta').absolute())

# 2. Modify the Configuration
# The default config expects a 'screen' modality. 
# If your export folder is named 'images' (from the previous step), 
# we need to copy the screen configuration to an 'images' key.
if 'screen' in cfg.dataset.modality_config:
    # Copy default screen config to 'images' key to match folder name
    cfg.dataset.modality_config.images = cfg.dataset.modality_config.screen
    # Optional: Remove 'screen' if it doesn't exist on disk to avoid warnings
    del cfg.dataset.modality_config.screen

# Update 'images' specific settings (formerly screen)
cfg.dataset.modality_config.images.include_blanks = True
# Ensure the valid_condition matches metadata you saved (e.g., you saved 'tiers'=['train'])
cfg.dataset.modality_config.images.valid_condition = {"tier": "train"}

# Update 'responses' settings if needed
if 'responses' in cfg.dataset.modality_config:
    # Adjust sampling rate if your exported data is already subsampled or different
    # cfg.dataset.modality_config.responses.sampling_rate = 30 
    pass

# Set dataloader parameters
cfg.dataloader.num_workers = 0 # Set to 0 for debugging/local tests, increase for speed later
cfg.dataloader.batch_size = 4

print("Loading Configuration:")
print(OmegaConf.to_yaml(cfg.dataset.modality_config))

# 3. Instantiate the Dataloader
# get_multisession_dataloader expects a list of paths
try:
    train_dl = get_multisession_dataloader([dataset_path], cfg)
    print(f"Successfully created dataloader with {len(train_dl)} batches.")

    # 4. Test loading a batch
    iterator = iter(train_dl)
    dataset_key, batch = next(iterator)

    print(f"\nLoaded batch from dataset: {dataset_key}")
    for key, value in batch.items():
        if hasattr(value, 'shape'):
            print(f"Modality: {key:<15} Shape: {value.shape}")
        else:
            print(f"Modality: {key:<15} Type: {type(value)}")

except Exception as e:
    print(f"\nError loading dataset: {e}")
    print("Ensure the exported folder contains the 'meta.yml' files required by Experanto.")

Loading Configuration:
responses:
  keep_nans: false
  sampling_rate: 8
  chunk_size: 16
  offset: 0.0
  transforms:
    normalization: standardize
  interpolation:
    interpolation_mode: nearest_neighbor
  filters:
    nan_filter:
      __target__: experanto.filters.common_filters.nan_filter
      __partial__: true
      vicinity: 0.05
eye_tracker:
  keep_nans: false
  sampling_rate: 30
  chunk_size: 60
  offset: 0
  transforms:
    normalization: normalize
  interpolation:
    interpolation_mode: nearest_neighbor
  filters:
    nan_filter:
      __target__: experanto.filters.common_filters.nan_filter
      __partial__: true
      vicinity: 0.05
treadmill:
  keep_nans: false
  sampling_rate: 30
  chunk_size: 60
  offset: 0
  transforms:
    normalization: normalize
  interpolation:
    interpolation_mode: nearest_neighbor
  filters:
    nan_filter:
      __target__: experanto.filters.common_filters.nan_filter
      __partial__: true
      vicinity: 0.05
images:
  keep_nans: false
  s